In [ ]:
%pip install yfiles_jupyter_graphs_for_neo4j --quiet
%pip install neo4j --quiet
from yfiles_jupyter_graphs_for_neo4j.Yfiles_Neo4j_Graphs import YfilesNeo4jGraphs
from neo4j import GraphDatabase
from datetime import datetime

NEO4J_URI      = "neo4j+ssc://demo.neo4jlabs.com" 
NEO4J_USERNAME = "movies"
NEO4J_PASSWORD = "movies"
driver = GraphDatabase.driver(uri = NEO4J_URI, auth = (NEO4J_USERNAME, NEO4J_PASSWORD), database = 'movies')

g = YfilesNeo4jGraphs(driver)

Configurable values for the node visualization: `textbinding`, `color`, `size`, `type`, `styles`, `scale_factor`, `position`, `layout`, ànd `property`

Configurable values for the relationship visualization: `textbinding`, `color`, `thickness` and `property`

When adding a configuration, you always need to specify the type of node or edge you want to adjust. (Here 'Person' or 'DIRECTED'),
additionally you can define the new values. For that you can either use:
- a function returning a value
- a constant value
- a property

In [ ]:
g.add_node_configuration('Person', text='name', size=(100,100), styles=lambda node : {'shape': 'rectangle'})
g.add_node_configuration('Movie', text= lambda node : {
        'text': datetime.now().strftime("%Y-%m-%d %H:%M:%S"), 
        'backgroundColor': '#5C268B', 
        'fontSize': 20, 
        'color': '#FF6EC7', 
        'position': 'north', 
        'maximumWidth': 130, 
        'wrapping': 'word', 
        'textAlignment': 'center'
    })  
g.add_relationship_configuration('DIRECTED', text='label', color="blue")
g.add_relationship_configuration('PRODUCED', text = lambda edge: 'produced', color= lambda edge: '#AC94F4')
g.add_relationship_configuration('ACTED_IN', thickness_factor=2, color='#AC94F4') 

In [ ]:
g.show_cypher("MATCH (s)-[r]->(t) RETURN s,r,t LIMIT 20")